## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-28-02-19-39 +0000,nypost,"Japan’s new PM donates 250 cherry trees, firew...",https://nypost.com/2025/10/27/us-news/japan-pm...
1,2025-10-28-02-11-44 +0000,startribune,Firefighters race to rescue man badly burned a...,https://www.startribune.com/firefighters-race-...
2,2025-10-28-02-02-54 +0000,startribune,Tributes grow after head-on crash in Chisago C...,https://www.startribune.com/tributes-grow-afte...
3,2025-10-28-02-02-00 +0000,wsj,Why ‘Yellowstone’ Mastermind Taylor Sheridan D...,https://www.wsj.com/business/media/why-yellows...
4,2025-10-28-02-00-00 +0000,wsj,Trump’s Big Tariff Task in Asia Is to Close th...,https://www.wsj.com/economy/trade/trump-tariff...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,51
1,new,19
87,china,18
343,he,14
155,milei,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
260,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...,145
298,2025-10-27-04-17-00 +0000,wsj,"Asian Markets Rise as U.S.-China Talks, Trump ...",https://www.wsj.com/finance/stocks/asian-marke...,109
44,2025-10-27-23-45-36 +0000,bbc,Trump does not rule out seeking third term - b...,https://www.bbc.com/news/articles/c797q57ple9o...,101
233,2025-10-27-12-47-34 +0000,cbc,When will Trump's tariff increase hit Canada? ...,https://www.cbc.ca/news/world/trump-tariff-can...,97
299,2025-10-27-04-09-01 +0000,nypost,"US, China talks sketch out rare earths, tariff...",https://nypost.com/2025/10/27/world-news/us-ch...,97


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
260,145,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...
268,79,2025-10-27-09-01-23 +0000,wsj,"Stock Market News, Oct. 27, 2025: Dow Hits Ne...",https://www.wsj.com/livecoverage/stock-market-...
41,54,2025-10-27-23-52-00 +0000,wsj,"“Both political parties have made their point,...",https://www.wsj.com/politics/policy/union-pres...
81,51,2025-10-27-21-54-59 +0000,nypost,Storm chaser captures stunning eye of Hurrican...,https://nypost.com/2025/10/27/world-news/hurri...
233,46,2025-10-27-12-47-34 +0000,cbc,When will Trump's tariff increase hit Canada? ...,https://www.cbc.ca/news/world/trump-tariff-can...
155,39,2025-10-27-17-47-00 +0000,wsj,Indiana Gov. Mike Braun called for a special l...,https://www.wsj.com/politics/policy/indiana-sp...
202,38,2025-10-27-14-59-00 +0000,wsj,Five Takeaways From Milei’s Surprising Victory...,https://www.wsj.com/world/americas/five-takeaw...
149,38,2025-10-27-18-01-17 +0000,nypost,Andrew Cuomo narrows gap against Zohran Mamdan...,https://nypost.com/2025/10/27/us-news/andrew-c...
27,32,2025-10-28-00-28-08 +0000,nypost,"Murder suspect who killed himself in jail, lin...",https://nypost.com/2025/10/27/us-news/possible...
277,31,2025-10-27-09-00-00 +0000,wsj,Democratic leaders have pinned their hopes on ...,https://www.wsj.com/politics/elections/these-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
